In [1]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import copy
from collections import defaultdict
from scipy.special import digamma, polygamma, loggamma

# **1. Preprocessing**

## **1-1. Load data**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# data path

train_path = "/content/drive/MyDrive/ML_lab/2_LDA/data/pos_tag/train.txt"
test_path = "/content/drive/MyDrive/ML_lab/2_LDA/data/pos_tag/test.txt"

In [4]:
def read_file(path):
  raw = open(path, 'r').read().split('\n') # 파일을 읽어와 줄마다 나누기
  data, doc = list(), list()

  for i, line in enumerate(raw):
    if line.strip(): # 빈 줄이 아니면
      word = line.split(' ')[0].lower() # 첫 번째 단어를 얻어온 후 소문자로 변환
      doc.append(word)
    else:
      data.append(doc) # EOD(End Of Document)
      doc = list()
  data.append(doc) # 맨 마지막 문서 추가

  return data

In [5]:
# 데이터 불러오기
train_data = read_file(train_path)
test_data = read_file(test_path)

# 맨 마지막 문서는 제거
del train_data[-1], test_data[-1]

## **1-2. Preprocess Data**

In [6]:
# some words that we do not want to consider
stopwords = [""," "]

In [7]:
## remove numbers, special characters, etc.
# only consider words with alphabet

def only_alphabet(corpus,stopword_list = stopwords):
  data = list()
  for doc in corpus:
    temp = list()
    for word in doc:
      word = re.sub('[^a-z]', '', word) # non-alphabet 문자 제거(=> 정규표현식 활용)
      if word not in stopwords:
        temp.append(word)
    data.append(temp)

  return data

In [8]:
train_data = only_alphabet(train_data)
test_data = only_alphabet(test_data)

In [9]:
## count word occurrence in corpus

def count_vocab(corpus):
  vocab = defaultdict(int) # dictionary 형태로 word occurence를 기록
  for doc in corpus:
    for word in doc:
      vocab[word] += 1

  return vocab

In [10]:
vocab_count = count_vocab(train_data)

In [11]:
## 특정 빈도수 이상인 단어들만 고려
# sparse한 단어는 무시

def vocab_top(vocab,cnt):
  temp = defaultdict(int)
  for voca, count in vocab.items():
    if count > cnt:
      temp[voca] = count

  return temp

In [12]:
vocab_count = vocab_top(vocab_count,cnt = 5)

In [13]:
## filter out low-occurrence words

def filter_vocab(corpus,vocab):
  data = list()
  for doc in corpus:
    temp = list()
    for word in doc:
      if word in vocab.keys():
        temp.append(word)
    if temp:
      data.append(temp)

  return data

In [14]:
train_data = filter_vocab(train_data,vocab_count)
test_data = filter_vocab(test_data,vocab_count)

In [15]:
## construct voca-index-matching dictionary

def voca_index(vocab):
  vocab_to_index, index_to_vocab = dict(), dict()
  for i, voca in enumerate(vocab.keys()):
    vocab_to_index[voca] = i
    index_to_vocab[i] = voca

  return vocab_to_index, index_to_vocab

In [16]:
v_t_i, i_t_v = voca_index(vocab_count)

In [17]:
## convert corpus-with-words to corpus-with-index

def corpus_to_index(corpus,vocab_to_index):
  data = list()
  for doc in corpus:
    temp = list()
    for word in doc:
      temp.append(vocab_to_index[word])
    data.append(temp)

  return data

In [18]:
train_data_idx = corpus_to_index(train_data,v_t_i)
test_data_idx = corpus_to_index(test_data,v_t_i)

# **2. Modeling**

In [19]:
class LDA_VI:
  def __init__(self,docs, num_topic=10, vocab = None, alpha=1.,num_iter=100,lr=1e-3):
    self.docs = docs
    self.num_topic = num_topic
    self.vocab = vocab
    self.num_vocab = len(self.vocab)
    self.num_docs = len(self.docs)

    # Initialize alpha and beta
    self.alpha = np.ones(self.num_topic)*(1/self.num_topic)
    self.beta = np.ones((self.num_topic, self.num_vocab))*(1/self.num_vocab)
    # beta 초기값 업데이트
    for i in range(self.num_topic):
      for v in range(self.num_vocab):
        self.beta[i][v] += np.random.rand(1) * 0.01
    # beta normailize 진행
    for i in range(self.num_topic):
      self.beta[i] /= np.sum(self.beta[i])

    # Initialize gamma and phi
    self.gamma = np.ones((self.num_docs,self.num_topic))*(1/self.num_topic)
    self.phi = np.ones((self.num_docs,self.num_topic,self.num_vocab))*(1/self.num_vocab)

    # Initialize z (topic assignments for words in documents)
    self.z = list()
    for doc in self.docs:
      temp = list()
      for word in doc:
        temp.append(None)
      self.z.append(temp)

    # Initialize parameters for variational inference
    self.num_iter = num_iter
    self.lr = lr

  def update_gamma(self, d):
    ## Update gamma (document-topic distributions)
    # d: document의 index
    self.gamma[d, :] = self.alpha + np.sum(self.phi[d, :, :], axis = 1)

  def update_phi(self, d, word):
    ## Update phi (topic-word distributions)
    for i in range(self.num_topic):
      self.phi[d, i, word] = self.beta[i, word] * np.exp(digamma(self.gamma[d, i]))
    # phi 정규화
    self.phi[d, :, word] /= np.sum(self.phi[d, :, word])

  def update_beta(self):
    ## Update beta(topic-word prior)
    for i in range(self.num_topic):
      for j in range(self.num_vocab):
        self.beta[i, j] = np.sum( [self.phi[d, i, j] for d in range(self.num_docs) for n in range(len(self.docs[d])) if self.docs[d][n] == j] )
    # beta 정규화
    self.beta /= np.sum(self.beta, axis=1)[:, np.newaxis]

  def update_alpha(self):
    ## Update alpha(document-topic prior)
    M = self.num_docs
    g = np.zeros(self.num_topic) # gradient
    H = np.zeros((self.num_topic, self.num_topic)) # Hessian matrix

    # Gradient 계산
    for i in range(self.num_topic):
      g[i] = M * ( digamma(np.sum(self.alpha)) - digamma(self.alpha[i]) ) + np.sum( digamma(self.gamma[:, i]) - digamma(np.sum(self.gamma, axis = 1)) )

    # Hessian 계산
    for i in range(self.num_topic):
      for j in range(self.num_topic):
        if i == j:
          H[i, j] = -M * polygamma(1, self.alpha[i])
        H[i, j] += M * polygamma(1, np.sum(self.alpha))

    # alpha 업데이트(=> Newton-Raphson)
    self.alpha -= np.linalg.inv(H).dot(g)

  def e_step(self):
    ## Perform E-step of the variational inference
    for d, doc in enumerate(self.docs):
      for n, word in enumerate(doc):
        self.update_phi(d, word)
      self.update_gamma(d)

  def m_step(self):
    ## Perform M-step of the variational inference
    self.update_beta()
    self.update_alpha()

  def compute_elbo(self):
    ## Compute Evidence Lower Bound (ELBO)

    # 필요한 파라미터 정의
    elbo = 0
    M = self.num_docs
    k = self.num_topic

    # 1번 수식
    for d in range(M): # document
      for n in range(len(self.docs[d])):
        j = self.docs[d][n] # word idx
        for i in range(k): # topic
          elbo += self.phi[d, i, n] * np.log(self.beta[i, j])

    # 2번 수식
    for d in range(M):
      for n in range(len(self.docs[d])):
        for i in range(k):
          elbo += self.phi[d, i, n] * (digamma(self.gamma[d, i]) - digamma(np.sum(self.gamma[d, :])))

    # 3번 수식
    elbo += M * (loggamma(np.sum(self.alpha)) - np.sum(loggamma(self.alpha)))

    for d in range(M):
      for i in range(k):
        elbo += (self.alpha[i] - 1) * (digamma(self.gamma[d, i]) - digamma(np.sum(self.gamma[d, :])))

    # 4번 수식
    for d in range(M):
      for i in range(k):
        elbo -= (self.gamma[d, i] - 1) * (digamma(self.gamma[d, i]) - digamma(np.sum(self.gamma[d, :])))

    for d in range(M):
      elbo -= loggamma(np.sum(self.gamma[d, :])) - np.sum(loggamma(self.gamma[d, :]))

    # 5번 수식
    for d in range(M):
      for n in range(len(self.docs[d])):
        for i in range(k):
          elbo -= self.phi[d, i, n] * np.log(self.phi[d, i, n])

    return elbo

  def compute_perplexity(self, elbo):
    # 분모
    N = sum(len(doc) for doc in self.docs)

    perplexity = np.exp(-elbo / N)
    return perplexity

  def run(self):
    ## Run the variational inference algorithm
    for iter in range(self.num_iter):
      print(f"=== Iteration: {iter} ===")
      self.e_step()
      self.m_step()
      print(f"Alpha: {self.alpha}")

      elbo = self.compute_elbo()
      print(f"ELBO: {elbo}")
      perplexity = self.compute_perplexity(elbo)
      print(f"Perplexity: {perplexity}")

    print("EM Algorithm 끝!")

# **3. Run!!**

In [20]:
model = LDA_VI(test_data_idx,num_topic = 10, vocab = v_t_i, alpha = 1.,num_iter = 50,lr = 1e-3)
model.run()

<ipython-input-19-9d8215f468c9>:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.beta[i][v] += np.random.rand(1) * 0.01


=== Iteration: 0 ===
Alpha: [0.1858029  0.18650543 0.18610183 0.18602988 0.18635968 0.18629456
 0.1866397  0.18667247 0.18639214 0.18662115]
ELBO: -49642.42292683816
Perplexity: 4.3879471706070685
=== Iteration: 1 ===
Alpha: [0.33644681 0.338539   0.33763257 0.33737339 0.33841885 0.33818936
 0.33874808 0.33917896 0.3383265  0.33877789]
ELBO: -42678.791662814365
Perplexity: 3.5658876118172786
=== Iteration: 2 ===
Alpha: [0.59271617 0.59595669 0.59604078 0.59538517 0.59726329 0.59673554
 0.59530785 0.59756153 0.59617879 0.59574952]
ELBO: -37458.50239421479
Perplexity: 3.0523114067015196
=== Iteration: 3 ===
Alpha: [1.01013495 1.01073839 1.01774891 1.01638988 1.01774987 1.0168694
 1.00599742 1.01365069 1.01318224 1.00786265]
ELBO: -33935.07959732787
Perplexity: 2.7481704591232288
=== Iteration: 4 ===
Alpha: [1.64364568 1.63260908 1.65850538 1.65606949 1.65292471 1.6518577
 1.61736485 1.63638059 1.64047734 1.62239736]
ELBO: -31902.18430302827
Perplexity: 2.5866791973876775
=== Iteration: 5

KeyboardInterrupt: 

In [ ]:
## ELBO 시각화



# **4. Inference**

In [ ]:
## analyze each doc's topic proportion

for d, gamma in enumerate(model.gamma):
  print(f"Document {d} topic distribution: {gamma / np.sum(gamma)}")

In [ ]:
## analyze topic-word proportion

for k, beta in enumerate(model.beta):
  top_words = np.argsort(beta)[-10:]
  print(f"Topic {k} top words: {[i_t_v[i] for i in top_words]}")